In [ ]:
# Inpaiting

import PIL
import torch
from diffusers import StableDiffusionInpaintPipeline

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

In [ ]:
import requests
from io import BytesIO
import PIL

def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

person = download_image("https://github.com/belonel/cloth-virtual-try-on-using-stable-diffusion/raw/main/readme_imgs/IMAGE_2023-02-17_125128.jpg")
person

In [ ]:
from transformers import pipeline

semantic_segmentation = pipeline("image-segmentation", "mattmdjaga/segformer_b2_clothes")
results = semantic_segmentation(person)
results

In [ ]:
mask_image = results[2]["mask"]
mask_image

In [ ]:
from PIL import Image
import numpy as np

def combine_masks(result_list, target_labels):
    result_mask = np.zeros(result_list[0]['mask'].size[::-1], dtype=np.uint8)
    
    for item in result_list:
        if item['label'] in target_labels:
            mask_array = np.array(item['mask'])
            result_mask = np.logical_or(result_mask, mask_array)
    
    return Image.fromarray(result_mask.astype(np.uint8) * 255)

# 대상 레이블 리스트
target_labels = ['Hat', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 
                 'Dress', 'Belt', 'Left-shoe', 'Right-shoe', 'Bag', 'Scarf']


combined_mask = combine_masks(results, target_labels)
combined_mask

In [ ]:
mask_image = combined_mask

In [ ]:
prompt = "Blue jump suits"
image = pipe(prompt=prompt, image=person, mask_image=mask_image).images[0]
image

In [ ]:
import gradio as gr
import PIL
import torch
from diffusers import StableDiffusionInpaintPipeline
import requests
from io import BytesIO
from transformers import pipeline

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

semantic_segmentation = pipeline("image-segmentation", "mattmdjaga/segformer_b2_clothes")

from PIL import Image
import numpy as np

def combine_masks(result_list, target_labels):
    result_mask = np.zeros(result_list[0]['mask'].size[::-1], dtype=np.uint8)
    
    for item in result_list:
        if item['label'] in target_labels:
            mask_array = np.array(item['mask'])
            result_mask = np.logical_or(result_mask, mask_array)
    
    return Image.fromarray(result_mask.astype(np.uint8) * 255)

target_labels = ['Hat', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 
                 'Dress', 'Belt', 'Left-shoe', 'Right-shoe', 'Bag', 'Scarf']

def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

def virtual_try_on(person_image, prompt):
    results = semantic_segmentation(person_image)
    mask_image = combine_masks(results, target_labels)
    
    output_image = pipe(prompt=prompt, image=person_image, mask_image=mask_image).images[0]
    
    return output_image

iface = gr.Interface(
    fn=virtual_try_on,
    inputs=[
        gr.Image(type="pil", label="Person Image"),
        gr.Textbox(label="Prompt (e.g., 'Stripe shirts')")
    ],
    outputs=gr.Image(type="pil", label="Result"),
    title="Virtual Try-On using Stable Diffusion",
    description="Upload a person image and enter a prompt to virtually try on the clothing."
)

iface.launch()